In [ ]:
#!pip install git+https://github.com/huggingface/transformers

In [1]:
from IPython.display import Audio

In [2]:
import torch
import torchaudio
from transformers import AutoProcessor, SeamlessM4Tv2Model

C:\Users\Administrator\anaconda3\envs\pdf\Lib\site-packages\transformers\deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [18]:
import pyaudio
import wave
import requests
import json

In [17]:
def record_audio(RECORD_SECONDS, WAVE_OUTPUT_FILENAME):
    #--------- SETTING PARAMS FOR OUR AUDIO FILE ------------#
    FORMAT = pyaudio.paInt16    # format of wave
    CHANNELS = 1                # no. of audio channels
    RATE = 44100                # frame rate
    CHUNK = 1024                # frames per audio sample
    #--------------------------------------------------------#
     
    # creating PyAudio object
    audio = pyaudio.PyAudio()
     
    # open a new stream for microphone
    # It creates a PortAudio Stream Wrapper class object
    stream = audio.open(format=FORMAT,channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)


    #----------------- start of recording -------------------#
    print("Listening...")

    # list to save all audio frames
    frames = []

    for i in range(int(RATE / CHUNK * RECORD_SECONDS)):
        # read audio stream from microphone
        data = stream.read(CHUNK)
        # append audio data to frames list
        frames.append(data)

    #------------------ end of recording --------------------#   
    print("Finished recording.")
      
    stream.stop_stream()    # stop the stream object
    stream.close()          # close the stream object
    audio.terminate()       # terminate PortAudio

    #------------------ saving audio ------------------------#

    # create wave file object
    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')

    # settings for wave file object
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))

    # closing the wave file object
    waveFile.close()

In [19]:
def read_audio(WAVE_FILENAME):
    # function to read audio(wav) file
    with open(WAVE_FILENAME, 'rb') as f:
        audio = f.read()
    return audio

In [20]:
# Wit speech API endpoint
API_ENDPOINT = 'https://api.wit.ai/speech'

# Wit.ai api access token
wit_access_token = 'S7JQYY2RA7ABGIFCOWGTUE6RDJSWHNF7'

In [21]:
text = ''

In [22]:
def RecognizeSpeech(AUDIO_FILENAME, num_seconds = 5):
    
    # record audio of specified length in specified audio file
    record_audio(num_seconds, AUDIO_FILENAME)
    
    # reading audio
    audio = read_audio(AUDIO_FILENAME)
    
    # defining headers for HTTP request
    headers = {'authorization': 'Bearer ' + wit_access_token,
               'Content-Type': 'audio/wav'}

    # making an HTTP post request
    resp = requests.post(API_ENDPOINT, headers = headers,
                         data = audio)
    
    # converting response content to JSON format
    
    data = json.loads(json.dumps(resp.text))
    data_text = str(data).splitlines()[-3].split(':')
    print(data_text[1])
    text = data_text[1]
#     print(data)
    # get text from data
#     text = data['text']
#     print("\nYou said: {}".format(text))
    # return the text
    return text

In [3]:
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
# from text
text_inputs = processor(text = RecognizeSpeech('myspeech.wav', 10) , src_lang="arb", return_tensors="pt")
audio_array_from_text = model.generate(**text_inputs, tgt_lang="eng")[0].cpu().numpy().squeeze()

Listening...
Finished recording.
 "عامل ايه يا باشا اخبارك? لك وحشة والله. بسم الله الرحمن الرحيم عليك",


In [24]:
Audio(audio_array_from_text, rate=16000)

In [25]:
output_tokens = model.generate(**text_inputs, tgt_lang="eng", generate_speech=False)
translated_text_from_text  = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)

In [26]:
translated_text_from_text

"What's the matter, Pasha? You're a monster and God. In the name of God, the merciful, the compassionate."